## Neural Network for multi-class classification of Walmart Kaggle

In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.metrics import log_loss

Using TensorFlow backend.
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
data = pd.read_csv('data/processed_data/processed_train.csv')
data2 = pd.read_csv('data/processed_data/processed_test.csv')

In [4]:
#Get features(X) and classes(Y) from data
Y = data['TripType']
X = data.drop(columns=['TripType'])
#X = data[data.columns[1:]].iloc[1:]
targets = list(np.unique(Y))
input_features = len(X.columns)
inp = len(data2.columns)
print(input_features, inp)
#X2 = data2[data2.columns[1:]].iloc[1:]
X2 = data2
print(X.columns)
print(data2.columns)
print(len(Y))

704 704
Index(['VisitNumber', 'Weekday', 'Upc', 'ScanCount', 'FinelineNumber',
       'FLCount', 'VisitFLCount', 'NumPurchases', 'NumReturns',
       'VisitNumPurchases',
       ...
       'UPC_7874235186.0', 'UPC_7874235187.0', 'UPC_7874235188.0',
       'UPC_7874235200.0', 'UPC_7874235201.0', 'UPC_7874235296.0',
       'UPC_7874298393.0', 'UPC_7976503128.0', 'UPC_8265750406.0',
       'UPC_9518801128.0'],
      dtype='object', length=704)
Index(['VisitNumber', 'Weekday', 'Upc', 'ScanCount', 'FinelineNumber',
       'FLCount', 'VisitFLCount', 'NumPurchases', 'NumReturns',
       'VisitNumPurchases',
       ...
       'UPC_7874235186.0', 'UPC_7874235187.0', 'UPC_7874235188.0',
       'UPC_7874235200.0', 'UPC_7874235201.0', 'UPC_7874235296.0',
       'UPC_7874298393.0', 'UPC_7976503128.0', 'UPC_8265750406.0',
       'UPC_9518801128.0'],
      dtype='object', length=704)
95674


In [5]:
#Create training / testing data using sklearn split for 67-33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, shuffle = True)

In [6]:
#Feature Scaling of the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X = sc.fit_transform(X)
X2 = sc.fit_transform(data2)

In [7]:
#Convert the strings to integers in classes (vector to matrix)
encoder = LabelEncoder()

#for 67-33 split train-->
encoder.fit(Y_train)
#print(encoder.classes_)
#print('----')
encoded_Y_train = encoder.transform(Y_train)
#print(encoded_Y_train[0:20], len(encoded_Y_train), '----------')
#print(encoder.inverse_transform(encoded_Y_train)[0:20], len(encoder.inverse_transform(encoded_Y_train)))

# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytrain = np_utils.to_categorical(encoded_Y_train)

#for 67-33 split test-->
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytest = np_utils.to_categorical(encoded_Y_test)


encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(dummy_y)


[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [8]:
classifier = Sequential()
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', 
                     activation = 'relu', input_dim = input_features))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))

#classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.25))

classifier.add(Dense(units = len(targets), kernel_initializer = 'uniform', activation = 'softmax'))
classifier.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, dummy_ytrain, validation_data=(X_test,dummy_ytest), epochs=100, batch_size = 4096)

Train on 64101 samples, validate on 31573 samples
Epoch 1/100
64101/64101 [==============================] - 9s 139us/step - loss: 3.1674 - acc: 0.1842 - val_loss: 2.5526 - val_acc: 0.3009
Epoch 2/100
64101/64101 [==============================] - 8s 119us/step - loss: 2.3436 - acc: 0.3781 - val_loss: 2.0171 - val_acc: 0.4737
Epoch 3/100
64101/64101 [==============================] - 7s 116us/step - loss: 1.9484 - acc: 0.4758 - val_loss: 1.6744 - val_acc: 0.5571
Epoch 4/100
64101/64101 [==============================] - 8s 129us/step - loss: 1.6946 - acc: 0.5358 - val_loss: 1.4851 - val_acc: 0.5970
Epoch 5/100
64101/64101 [==============================] - 9s 136us/step - loss: 1.5442 - acc: 0.5690 - val_loss: 1.3541 - val_acc: 0.6130
Epoch 6/100
64101/64101 [==============================] - 8s 118us/step - loss: 1.4230 - acc: 0.5975 - val_loss: 1.2881 - val_acc: 0.6223
Epoch 7/100
64101/64101 [==============================] - 9s 138us/step - loss: 1.3530 - acc: 0.6134 - val_loss: 1.

64101/64101 [==============================] - 8s 121us/step - loss: 0.7315 - acc: 0.7537 - val_loss: 0.8638 - val_acc: 0.7210
Epoch 60/100
64101/64101 [==============================] - 7s 109us/step - loss: 0.7290 - acc: 0.7556 - val_loss: 0.8643 - val_acc: 0.7205
Epoch 61/100
64101/64101 [==============================] - 7s 105us/step - loss: 0.7274 - acc: 0.7564 - val_loss: 0.8640 - val_acc: 0.7230
Epoch 62/100
64101/64101 [==============================] - 8s 123us/step - loss: 0.7183 - acc: 0.7575 - val_loss: 0.8603 - val_acc: 0.7222
Epoch 63/100
64101/64101 [==============================] - 7s 113us/step - loss: 0.7173 - acc: 0.7566 - val_loss: 0.8616 - val_acc: 0.7189
Epoch 64/100
64101/64101 [==============================] - 8s 118us/step - loss: 0.7170 - acc: 0.7578 - val_loss: 0.8601 - val_acc: 0.7219
Epoch 65/100
64101/64101 [==============================] - 9s 134us/step - loss: 0.7089 - acc: 0.7594 - val_loss: 0.8658 - val_acc: 0.7212
Epoch 66/100
64101/64101 [=======

In [9]:
classifier.fit(X, dummy_y, epochs=75, batch_size = 4096)

#y_pred = classifier.predict(X_test)
y_pred = classifier.predict(X2)

##score = classifier.evaluate(X_test, dummy_ytest, batch_size = 64)
##print(score)

#print (log_loss(Y_test, y_pred, labels = targets))


Epoch 1/75
95674/95674 [==============================] - 10s 105us/step - loss: 0.7646 - acc: 0.7513
Epoch 2/75
95674/95674 [==============================] - 11s 118us/step - loss: 0.7478 - acc: 0.7512
Epoch 3/75
95674/95674 [==============================] - 10s 109us/step - loss: 0.7375 - acc: 0.7549
Epoch 4/75
95674/95674 [==============================] - 11s 112us/step - loss: 0.7285 - acc: 0.7579
Epoch 5/75
95674/95674 [==============================] - 11s 113us/step - loss: 0.7221 - acc: 0.7586
Epoch 6/75
95674/95674 [==============================] - 10s 108us/step - loss: 0.7161 - acc: 0.7602
Epoch 7/75
95674/95674 [==============================] - 10s 107us/step - loss: 0.7146 - acc: 0.7590
Epoch 8/75
95674/95674 [==============================] - 10s 100us/step - loss: 0.7071 - acc: 0.7612
Epoch 9/75
95674/95674 [==============================] - 9s 92us/step - loss: 0.7025 - acc: 0.7610
Epoch 10/75
95674/95674 [==============================] - 9s 91us/step - loss: 0.70

In [10]:
print(y_pred)
print (type(y_pred))
print (len(y_pred[0]))

[[  6.89548961e-06   1.09950757e-04   2.06757424e-04 ...,   2.01317924e-03
    1.46419159e-04   1.42216112e-03]
 [  1.73657807e-03   8.07945500e-04   7.60952884e-04 ...,   7.76012912e-02
    5.26812859e-03   3.98885831e-02]
 [  8.17905820e-05   1.91040300e-07   3.20932614e-08 ...,   1.00662508e-11
    1.43495564e-14   9.96635258e-01]
 ..., 
 [  9.13401891e-06   7.13556787e-08   3.64610964e-09 ...,   4.82938889e-09
    1.12057559e-11   1.03561988e-03]
 [  4.11648438e-10   1.27261535e-10   4.35714714e-10 ...,   9.62975668e-04
    3.89640639e-03   5.46565570e-06]
 [  3.95282418e-11   5.49720495e-18   2.16474720e-14 ...,   2.33249131e-09
    2.01140837e-09   5.38856568e-07]]
<class 'numpy.ndarray'>
38


In [11]:
targets = list(np.unique(Y))
visits = data2.VisitNumber.values #data2 is test dataframe
targets = ["TripType_" + str(int(i)) for i in targets]
out = pd.DataFrame(y_pred, columns = targets, index=None) #y_pred is predicted probs
out.insert(0,'VisitNumber',visits)
out.to_csv("output7_nn.csv",index=False)

In [13]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.decomposition import TruncatedSVD

In [14]:
# Trucated SVD
#Get features(X) and classes(Y) from data
Y = data['TripType']
X = data.drop(columns=['TripType'])

svd = TruncatedSVD(n_components=200, n_iter=7, random_state=42)
svd = svd.fit(X) 
X = svd.transform(X)

In [15]:
#Create training / testing data using sklearn split for 67-33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, shuffle = True)

In [16]:
#Feature Scaling of the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X = sc.fit_transform(X)
X2 = sc.fit_transform(svd.transform(data2))

In [17]:
#Convert the strings to integers in classes (vector to matrix)
encoder = LabelEncoder()

#for 67-33 split train-->
encoder.fit(Y_train)
#print(encoder.classes_)
#print('----')
encoded_Y_train = encoder.transform(Y_train)
#print(encoded_Y_train[0:20], len(encoded_Y_train), '----------')
#print(encoder.inverse_transform(encoded_Y_train)[0:20], len(encoder.inverse_transform(encoded_Y_train)))

# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytrain = np_utils.to_categorical(encoded_Y_train)

#for 67-33 split test-->
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytest = np_utils.to_categorical(encoded_Y_test)


encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(dummy_y)


[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [20]:
classifier = Sequential()
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', 
                     activation = 'relu', input_dim = 200))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))

#classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.25))

classifier.add(Dense(units = len(targets), kernel_initializer = 'uniform', activation = 'softmax'))
classifier.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, dummy_ytrain, validation_data=(X_test,dummy_ytest), epochs=100, batch_size = 4096)

Train on 64101 samples, validate on 31573 samples
Epoch 1/100
64101/64101 [==============================] - 5s 71us/step - loss: 3.3667 - acc: 0.0769 - val_loss: 3.0720 - val_acc: 0.2169
Epoch 2/100
64101/64101 [==============================] - 3s 53us/step - loss: 2.8272 - acc: 0.2949 - val_loss: 2.3613 - val_acc: 0.4498
Epoch 3/100
64101/64101 [==============================] - 4s 66us/step - loss: 2.1364 - acc: 0.4515 - val_loss: 1.8144 - val_acc: 0.5313
Epoch 4/100
64101/64101 [==============================] - 4s 62us/step - loss: 1.7624 - acc: 0.5248 - val_loss: 1.5335 - val_acc: 0.5775
Epoch 5/100
64101/64101 [==============================] - 3s 51us/step - loss: 1.5538 - acc: 0.5653 - val_loss: 1.3740 - val_acc: 0.6033
Epoch 6/100
64101/64101 [==============================] - 4s 61us/step - loss: 1.4183 - acc: 0.5946 - val_loss: 1.2498 - val_acc: 0.6370
Epoch 7/100
64101/64101 [==============================] - 4s 56us/step - loss: 1.3170 - acc: 0.6200 - val_loss: 1.1703 - 

64101/64101 [==============================] - 3s 52us/step - loss: 0.8024 - acc: 0.7309 - val_loss: 0.8411 - val_acc: 0.7186
Epoch 60/100
64101/64101 [==============================] - 3s 51us/step - loss: 0.7963 - acc: 0.7333 - val_loss: 0.8427 - val_acc: 0.7163
Epoch 61/100
64101/64101 [==============================] - 3s 51us/step - loss: 0.7996 - acc: 0.7315 - val_loss: 0.8408 - val_acc: 0.7184
Epoch 62/100
64101/64101 [==============================] - 3s 51us/step - loss: 0.7938 - acc: 0.7347 - val_loss: 0.8411 - val_acc: 0.7158
Epoch 63/100
64101/64101 [==============================] - 3s 52us/step - loss: 0.7939 - acc: 0.7319 - val_loss: 0.8377 - val_acc: 0.7195
Epoch 64/100
64101/64101 [==============================] - 3s 51us/step - loss: 0.7929 - acc: 0.7341 - val_loss: 0.8366 - val_acc: 0.7177
Epoch 65/100
64101/64101 [==============================] - 4s 56us/step - loss: 0.7895 - acc: 0.7356 - val_loss: 0.8348 - val_acc: 0.7192
Epoch 66/100
64101/64101 [==============

In [21]:
classifier.fit(X, dummy_y, epochs=75, batch_size = 4096)

#y_pred = classifier.predict(X_test)
y_pred = classifier.predict(svd.transform(X2))

##score = classifier.evaluate(X_test, dummy_ytest, batch_size = 64)
##print(score)

#print (log_loss(Y_test, y_pred, labels = targets))


Epoch 1/100
95674/95674 [==============================] - 4s 46us/step - loss: 0.8030 - acc: 0.7301
Epoch 2/100
95674/95674 [==============================] - 4s 45us/step - loss: 0.7960 - acc: 0.7317
Epoch 3/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.7896 - acc: 0.7330
Epoch 4/100
95674/95674 [==============================] - 5s 51us/step - loss: 0.7854 - acc: 0.7344
Epoch 5/100
95674/95674 [==============================] - 5s 50us/step - loss: 0.7806 - acc: 0.7361
Epoch 6/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.7784 - acc: 0.7377
Epoch 7/100
95674/95674 [==============================] - 5s 50us/step - loss: 0.7762 - acc: 0.7361
Epoch 8/100
95674/95674 [==============================] - 5s 54us/step - loss: 0.7729 - acc: 0.7380
Epoch 9/100
95674/95674 [==============================] - 5s 50us/step - loss: 0.7725 - acc: 0.7358
Epoch 10/100
95674/95674 [==============================] - 4s 45us/step - loss: 0.7670 - a

95674/95674 [==============================] - 5s 49us/step - loss: 0.6838 - acc: 0.7616
Epoch 82/100
95674/95674 [==============================] - 5s 50us/step - loss: 0.6815 - acc: 0.7602
Epoch 83/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.6832 - acc: 0.7616
Epoch 84/100
95674/95674 [==============================] - 4s 46us/step - loss: 0.6778 - acc: 0.7618
Epoch 85/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.6783 - acc: 0.7618
Epoch 86/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.6778 - acc: 0.7631
Epoch 87/100
95674/95674 [==============================] - 6s 59us/step - loss: 0.6775 - acc: 0.7632
Epoch 88/100
95674/95674 [==============================] - 5s 48us/step - loss: 0.6778 - acc: 0.7620
Epoch 89/100
95674/95674 [==============================] - 5s 49us/step - loss: 0.6794 - acc: 0.7618
Epoch 90/100
95674/95674 [==============================] - 5s 47us/step - loss: 0.6756 - acc: 

ValueError: Error when checking : expected dense_7_input to have shape (200,) but got array with shape (704,)

In [23]:
print(y_pred)
print (type(y_pred))
print (len(y_pred[0]))

[[  6.76374329e-05   2.61327077e-04   1.00500917e-03 ...,   1.26903623e-01
    9.59358811e-02   2.96167244e-04]
 [  2.12006751e-04   6.65093306e-04   1.93292543e-03 ...,   1.92518175e-01
    2.47157905e-02   1.42911803e-02]
 [  2.56727315e-08   5.58559655e-13   3.56907407e-12 ...,   5.07011798e-14
    1.09436136e-17   9.99942899e-01]
 ..., 
 [  5.91528396e-06   1.75345623e-11   1.21378634e-12 ...,   2.26158603e-09
    6.18105447e-13   4.04347520e-04]
 [  1.07475896e-10   2.46216159e-10   2.99787972e-09 ...,   1.74550582e-02
    3.51560628e-03   2.99923408e-06]
 [  2.03383888e-09   6.87553703e-12   4.40295267e-09 ...,   1.81319319e-05
    3.21555417e-05   2.26552438e-04]]
<class 'numpy.ndarray'>
38


In [24]:
targets = list(np.unique(Y))
visits = data2.VisitNumber.values #data2 is test dataframe
targets = ["TripType_" + str(int(i)) for i in targets]
out = pd.DataFrame(y_pred, columns = targets, index=None) #y_pred is predicted probs
out.insert(0,'VisitNumber',visits)
out.to_csv("output/output7_nn_svd.csv",index=False)

In [25]:
#Get features(X) and classes(Y) from data
Y = data['TripType']
X = data.drop(columns=['TripType'])

#ANOVA F-value
skb = SelectKBest(k = 200)
skb = skb.fit(X, Y)
names = X.columns.values[skb.get_support()] #names of the columns of X whose features are retained
X = skb.transform(X)

/usr/local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 11  13  14  16 111 123 133 167 191 201] are constant.
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [31]:
#Create training / testing data using sklearn split for 67-33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, shuffle = True)

In [36]:
#Feature Scaling of the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X = sc.fit_transform(X)
X2 = sc.fit_transform(skb.transform(data2))

In [33]:
#Convert the strings to integers in classes (vector to matrix)
encoder = LabelEncoder()

#for 67-33 split train-->
encoder.fit(Y_train)
#print(encoder.classes_)
#print('----')
encoded_Y_train = encoder.transform(Y_train)
#print(encoded_Y_train[0:20], len(encoded_Y_train), '----------')
#print(encoder.inverse_transform(encoded_Y_train)[0:20], len(encoder.inverse_transform(encoded_Y_train)))

# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytrain = np_utils.to_categorical(encoded_Y_train)

#for 67-33 split test-->
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytest = np_utils.to_categorical(encoded_Y_test)


encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(dummy_y)


[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [34]:
classifier = Sequential()
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', 
                     activation = 'relu', input_dim = 200))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))

#classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.25))

classifier.add(Dense(units = len(targets), kernel_initializer = 'uniform', activation = 'softmax'))
classifier.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, dummy_ytrain, validation_data=(X_test,dummy_ytest), epochs=100, batch_size = 4096)

Train on 64101 samples, validate on 31573 samples
Epoch 1/100
64101/64101 [==============================] - 5s 72us/step - loss: 3.2826 - acc: 0.2041 - val_loss: 2.6368 - val_acc: 0.3310
Epoch 2/100
64101/64101 [==============================] - 3s 52us/step - loss: 2.3146 - acc: 0.4025 - val_loss: 1.9163 - val_acc: 0.5095
Epoch 3/100
64101/64101 [==============================] - 4s 60us/step - loss: 1.8323 - acc: 0.5125 - val_loss: 1.6044 - val_acc: 0.5614
Epoch 4/100
64101/64101 [==============================] - 3s 51us/step - loss: 1.6043 - acc: 0.5576 - val_loss: 1.4115 - val_acc: 0.6039
Epoch 5/100
64101/64101 [==============================] - 3s 53us/step - loss: 1.4568 - acc: 0.5911 - val_loss: 1.2971 - val_acc: 0.6295
Epoch 6/100
64101/64101 [==============================] - 3s 54us/step - loss: 1.3678 - acc: 0.6128 - val_loss: 1.2467 - val_acc: 0.6308
Epoch 7/100
64101/64101 [==============================] - 3s 51us/step - loss: 1.3148 - acc: 0.6229 - val_loss: 1.1763 - 

64101/64101 [==============================] - 3s 51us/step - loss: 0.8849 - acc: 0.7058 - val_loss: 0.8548 - val_acc: 0.7122
Epoch 60/100
64101/64101 [==============================] - 4s 58us/step - loss: 0.8790 - acc: 0.7069 - val_loss: 0.8552 - val_acc: 0.7110
Epoch 61/100
64101/64101 [==============================] - 4s 55us/step - loss: 0.8774 - acc: 0.7085 - val_loss: 0.8515 - val_acc: 0.7124
Epoch 62/100
64101/64101 [==============================] - 4s 57us/step - loss: 0.8755 - acc: 0.7093 - val_loss: 0.8524 - val_acc: 0.7124
Epoch 63/100
64101/64101 [==============================] - 4s 55us/step - loss: 0.8751 - acc: 0.7074 - val_loss: 0.8495 - val_acc: 0.7135
Epoch 64/100
64101/64101 [==============================] - 4s 60us/step - loss: 0.8702 - acc: 0.7096 - val_loss: 0.8491 - val_acc: 0.7131
Epoch 65/100
64101/64101 [==============================] - 4s 60us/step - loss: 0.8679 - acc: 0.7105 - val_loss: 0.8471 - val_acc: 0.7150
Epoch 66/100
64101/64101 [==============

In [37]:
classifier.fit(X, dummy_y, epochs=75, batch_size = 4096)

#y_pred = classifier.predict(X_test)
y_pred = classifier.predict(X2)

##score = classifier.evaluate(X_test, dummy_ytest, batch_size = 64)
##print(score)

#print (log_loss(Y_test, y_pred, labels = targets))


In [38]:
print(y_pred)
print (type(y_pred))
print (len(y_pred[0]))

[[  1.41193930e-04   2.38921348e-04   1.66957572e-04 ...,   4.01559111e-04
    2.78175594e-05   1.95693746e-02]
 [  2.25480660e-04   1.63357501e-04   2.85220085e-05 ...,   1.63690932e-02
    9.68142296e-04   3.57328840e-02]
 [  1.28925658e-05   2.07943231e-07   2.49065302e-08 ...,   1.20248941e-11
    5.48838969e-13   9.87099588e-01]
 ..., 
 [  1.38571762e-04   1.68466840e-06   6.60230697e-08 ...,   8.72840758e-07
    2.07964082e-08   5.43073425e-03]
 [  2.00513384e-09   7.60494168e-10   6.84166057e-09 ...,   5.76538499e-03
    1.59583967e-02   3.86177882e-04]
 [  6.21060690e-06   3.16861133e-08   4.82383484e-06 ...,   1.59193238e-04
    5.75031270e-04   6.50571613e-03]]
<class 'numpy.ndarray'>
38


In [39]:
targets = list(np.unique(Y))
visits = data2.VisitNumber.values #data2 is test dataframe
targets = ["TripType_" + str(int(i)) for i in targets]
out = pd.DataFrame(y_pred, columns = targets, index=None) #y_pred is predicted probs
out.insert(0,'VisitNumber',visits)
out.to_csv("output7_nn_f_anova.csv",index=False)

In [40]:
#Get features(X) and classes(Y) from data
Y = data['TripType']
X = data.drop(columns=['TripType'])

#Mutual Information
skb = SelectKBest(k = 200, score_func = mutual_info_classif)
skb = skb.fit(X, Y)
names = X.columns.values[skb.get_support()] #names of the columns of X whose features are retained
X = skb.transform(X)

In [41]:
#Create training / testing data using sklearn split for 67-33
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, shuffle = True)

In [42]:
#Feature Scaling of the data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X = sc.fit_transform(X)
X2 = sc.fit_transform(skb.transform(data2))

In [43]:
#Convert the strings to integers in classes (vector to matrix)
encoder = LabelEncoder()

#for 67-33 split train-->
encoder.fit(Y_train)
#print(encoder.classes_)
#print('----')
encoded_Y_train = encoder.transform(Y_train)
#print(encoded_Y_train[0:20], len(encoded_Y_train), '----------')
#print(encoder.inverse_transform(encoded_Y_train)[0:20], len(encoder.inverse_transform(encoded_Y_train)))

# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytrain = np_utils.to_categorical(encoded_Y_train)

#for 67-33 split test-->
encoder.fit(Y_test)
encoded_Y_test = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_ytest = np_utils.to_categorical(encoded_Y_test)


encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

print(dummy_y)


[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [45]:
classifier = Sequential()
classifier.add(Dense(units = 512, kernel_initializer = 'uniform', 
                     activation = 'relu', input_dim = 200))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))

#classifier.add(Dense(units = 128, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(0.25))

classifier.add(Dense(units = len(targets), kernel_initializer = 'uniform', activation = 'softmax'))
classifier.compile(optimizer = 'Adadelta', loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, dummy_ytrain, validation_data=(X_test,dummy_ytest), epochs=100, batch_size = 4096)

Train on 64101 samples, validate on 31573 samples
Epoch 1/100
64101/64101 [==============================] - 4s 64us/step - loss: 3.2340 - acc: 0.2093 - val_loss: 2.5864 - val_acc: 0.3252
Epoch 2/100
64101/64101 [==============================] - 3s 52us/step - loss: 2.3144 - acc: 0.3950 - val_loss: 1.9779 - val_acc: 0.4829
Epoch 3/100
64101/64101 [==============================] - 3s 51us/step - loss: 1.8732 - acc: 0.4968 - val_loss: 1.6548 - val_acc: 0.5533
Epoch 4/100
64101/64101 [==============================] - 3s 51us/step - loss: 1.6403 - acc: 0.5513 - val_loss: 1.4711 - val_acc: 0.5932
Epoch 5/100
64101/64101 [==============================] - 3s 52us/step - loss: 1.5043 - acc: 0.5800 - val_loss: 1.3770 - val_acc: 0.6069
Epoch 6/100
64101/64101 [==============================] - 3s 48us/step - loss: 1.4129 - acc: 0.5988 - val_loss: 1.3049 - val_acc: 0.6157
Epoch 7/100
64101/64101 [==============================] - 3s 49us/step - loss: 1.3404 - acc: 0.6161 - val_loss: 1.2234 - 

64101/64101 [==============================] - 4s 55us/step - loss: 0.8865 - acc: 0.7070 - val_loss: 0.8850 - val_acc: 0.7074
Epoch 60/100
64101/64101 [==============================] - 3s 54us/step - loss: 0.8877 - acc: 0.7084 - val_loss: 0.8839 - val_acc: 0.7091
Epoch 61/100
64101/64101 [==============================] - 3s 49us/step - loss: 0.8831 - acc: 0.7066 - val_loss: 0.8819 - val_acc: 0.7081
Epoch 62/100
64101/64101 [==============================] - 3s 49us/step - loss: 0.8757 - acc: 0.7086 - val_loss: 0.8804 - val_acc: 0.7080
Epoch 63/100
64101/64101 [==============================] - 3s 51us/step - loss: 0.8782 - acc: 0.7077 - val_loss: 0.8779 - val_acc: 0.7104
Epoch 64/100
64101/64101 [==============================] - 3s 49us/step - loss: 0.8753 - acc: 0.7089 - val_loss: 0.8802 - val_acc: 0.7084
Epoch 65/100
64101/64101 [==============================] - 3s 49us/step - loss: 0.8724 - acc: 0.7095 - val_loss: 0.8755 - val_acc: 0.7106
Epoch 66/100
64101/64101 [==============

In [46]:
classifier.fit(X, dummy_y, epochs=75, batch_size = 4096)

#y_pred = classifier.predict(X_test)
y_pred = classifier.predict(X2)

##score = classifier.evaluate(X_test, dummy_ytest, batch_size = 64)
##print(score)

#print (log_loss(Y_test, y_pred, labels = targets))


Epoch 1/75
95674/95674 [==============================] - 9s 91us/step - loss: 0.8633 - acc: 0.7128
Epoch 2/75
95674/95674 [==============================] - 8s 82us/step - loss: 0.8590 - acc: 0.7119
Epoch 3/75
95674/95674 [==============================] - 8s 80us/step - loss: 0.8546 - acc: 0.7134
Epoch 4/75
95674/95674 [==============================] - 7s 76us/step - loss: 0.8499 - acc: 0.7144
Epoch 5/75
95674/95674 [==============================] - 8s 80us/step - loss: 0.8518 - acc: 0.7132
Epoch 6/75
95674/95674 [==============================] - 7s 78us/step - loss: 0.8494 - acc: 0.7159
Epoch 7/75
95674/95674 [==============================] - 7s 68us/step - loss: 0.8452 - acc: 0.7149
Epoch 8/75
95674/95674 [==============================] - 8s 79us/step - loss: 0.8433 - acc: 0.7160
Epoch 9/75
95674/95674 [==============================] - 7s 74us/step - loss: 0.8390 - acc: 0.7175
Epoch 10/75
95674/95674 [==============================] - 7s 73us/step - loss: 0.8370 - acc: 0.7178

In [47]:
print(y_pred)
print (type(y_pred))
print (len(y_pred[0]))

[[  1.13561309e-05   4.84897791e-05   6.86969943e-05 ...,   1.67121191e-03
    4.11342226e-05   8.60248506e-03]
 [  1.56692264e-03   4.52379572e-05   2.83967984e-05 ...,   1.77219044e-02
    1.10528804e-03   3.93831432e-02]
 [  2.63817742e-06   3.93626678e-08   1.19925248e-09 ...,   4.67315726e-12
    2.01823606e-12   9.93685186e-01]
 ..., 
 [  2.02175870e-04   1.78521191e-06   4.58461713e-08 ...,   1.54845020e-06
    1.16178633e-07   5.08883875e-03]
 [  1.03808329e-09   4.32542135e-09   5.70667114e-09 ...,   7.56094512e-03
    1.46945743e-02   8.71775846e-05]
 [  3.08637209e-06   4.56720328e-10   8.78196886e-07 ...,   1.55533980e-05
    4.47973944e-05   2.93651386e-03]]
<class 'numpy.ndarray'>
38


In [48]:
targets = list(np.unique(Y))
visits = data2.VisitNumber.values #data2 is test dataframe
targets = ["TripType_" + str(int(i)) for i in targets]
out = pd.DataFrame(y_pred, columns = targets, index=None) #y_pred is predicted probs
out.insert(0,'VisitNumber',visits)
out.to_csv("output7_nn_mutual_info.csv",index=False)